In [1]:
import pandas as pd
import numpy as np
from string import punctuation
from fuzzywuzzy import fuzz
from nltk import word_tokenize

C:\Users\User\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('result_tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [3]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)

In [4]:
df = df.dropna()

In [5]:
df = df.drop_duplicates()

In [6]:
df = df.reset_index(drop=True)

In [7]:
df.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь
3,"«Сачхере» — «Абердин» И тот, и другой клуб оде...",1.45,Айнтрахт - 3 и более гола,Лига Ставок,2019-07-25 17:00:00,Лига Европы УЕФА,Флора,Айнтрахт Франкфурт
4,Я предполагаю ничью в этом матче с забитыми мя...,5,Арсенал забьет первым и не выиграет,1xСтавка,2019-05-09 19:00:00,Лига Европы УЕФА,Валенсия,Арсенал


### Нечеткий поиск по командам 1

In [8]:
%%time
tokens = []
teams1_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            if fuzz.ratio(df['team_of_season_1_name_in_russian'][i], t) > 70:
                teams1_match[i] = 1

Wall time: 2h 35min 19s


In [9]:
df['fuzzy_match_for_teams1'] = teams1_match

In [10]:
all_vals = df['fuzzy_match_for_teams1'].shape[0]
print(all_vals)

702908


In [11]:
success = df[df['fuzzy_match_for_teams1'] == 1].shape[0]
print(success)

330689


In [12]:
fuzzyline_team1 = success / all_vals
print(fuzzyline_team1)

0.47045843837315837


### Нечеткий поиск по командам 2

In [13]:
%%time
tokens = []
teams2_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            if fuzz.ratio(df['team_of_season_2_name_in_russian'][i], t) > 70:
                teams2_match[i] = 1

Wall time: 2h 30min 34s


In [14]:
df['fuzzy_match_for_teams2'] = teams2_match

In [27]:
all_vals = df['fuzzy_match_for_teams2'].shape[0]
print(all_vals)

702908


In [28]:
success = df[df['fuzzy_match_for_teams2'] == 1].shape[0]
print(success)

321167


In [29]:
fuzzyline_team2 = success / all_vals

In [30]:
print(fuzzyline_team2)

0.4569118575972958


### Нечеткий поиск по лигам

С использованием триграммов

In [18]:
%%time 
leagues_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'][:10].values):
    #print(i, text)
    words = word_tokenize(text)
    N = 3
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(grams)
    for gram in grams: 
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if fuzz.ratio(df['league_name_in_russian'][i], joined_gram) > 60: #можно регулировать score 
            #print(df['league_name_in_russian'][i], joined_gram)
            leagues_match[i] = 1

Wall time: 1.19 s


In [26]:
leagues_match[:15]

array([1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0.])

In [19]:
df['fuzzy_match_for_leagues'] = leagues_match

In [20]:
all_vals = df['fuzzy_match_for_leagues'].shape[0]
print(all_vals)
success = df[df['fuzzy_match_for_leagues'] == 1].shape[0]
print(success)
fuzzyline_leagues = success / all_vals
print(fuzzyline_leagues)

702908
7
9.958629009770838e-06


### Нечеткий поиск по букмекерам

Вариант без нграммов

In [31]:
%%time
tokens = []
bookmaker_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for text in txt_tokens:
        for t in text:
            if fuzz.ratio(df['bookmaker_name'][i], t) > 70:
                bookmaker_match[i] = 1

Wall time: 2h 24min 46s


In [32]:
df['fuzzy_match_for_bookmakers1'] = bookmaker_match

In [23]:
all_vals = df['fuzzy_match_for_bookmakers1'].shape[0]
print(all_vals)
success = df[df['fuzzy_match_for_bookmakers1'] == 1].shape[0]
print(success)
fuzzyline_bookmakers = success / all_vals
print(fuzzyline_bookmakers)

702908
1596
0.002270567414227751


Вариант с биграммами

In [33]:
%%time 
bookmaker_match = np.zeros(df.shape[0])
for i, text in enumerate(df['result.tsv'][:10].values):
    #print(i, text)
    words = word_tokenize(text)
    N = 2 
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(grams)
    for gram in grams: 
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if fuzz.ratio(df['bookmaker_name'][i], joined_gram) > 70: #можно регулировать score 
            #print(df['bookmaker_name'][i], joined_gram)
            bookmaker_match[i] = 1

Wall time: 761 ms


In [34]:
df['fuzzy_match_for_bookmakers2'] = bookmaker_match

In [35]:
all_vals = df['fuzzy_match_for_bookmakers2'].shape[0]
print(all_vals)
success = df[df['fuzzy_match_for_bookmakers2'] == 1].shape[0]
print(success)
fuzzyline_bookmakers = success / all_vals
print(fuzzyline_bookmakers)

702908
0
0.0
